# Useless Trump Tweet Generator
This is a trump tweet generator based on Jeremy Howard's great MOOC at fast.ai.  Go watch it now!

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [4]:
path = get_file('trump.csv', origin="https://github.com/bpb27/trump-tweet-archive/raw/master/data/realdonaldtrump/realdonaldtrump.csv")
import pandas as pd
text_pd = pd.read_csv (path).as_matrix()[:,2]
text_pd = [s.replace("&amp;", "&") for s in text_pd]
text = "\n".join(text_pd)
print('corpus length:', len(text))

corpus length: 3306584


In [5]:
print (text_pd[1000:1012])

['I will be interviewed from Cleveland, Ohio, on @seanhannity - Tonight at 10:00 P.M. Enjoy!', '"@ThAllenSBoucher: @DiamondandSilk @realDonaldTrump @seanhannity I love those beautiful gals." D + S = Two amazing women!', '.@YoungDems4Trump  Thank you!', 'Great new polls! Thank you Nevada, North Carolina & Ohio. Join the MOVEMENT today & lets #MAGA!\xe2\x80\xa6 https://t.co/Y8Sb8MNyXA', 'Thank you Toledo, Ohio! It is so important for you to get out and VOTE on November 8, 2016! Lets MAKE AMERICA SAFE\xe2\x80\xa6 https://t.co/MQdp4GgLIE', 'RT @GMA: WATCH: @IvankaTrump on "women who work;" empowering campaign celebrates modern women.  https://t.co/rMFe9o6WcL', 'Hopefully the violence & unrest in Charlotte will come to an immediate end. To those injured, get well soon. We need unity & leadership.', 'The situations in Tulsa and Charlotte are tragic. We must come together to make America safe again.', 'It is a MOVEMENT - not a campaign. Leaving the past behind, changing our future. Together, 

In [6]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [7]:
chars_ = sorted(list(set(text)))
vocab_size = len(chars_)+1
print('total chars:', vocab_size)

total chars: 165


In [8]:
chars_.insert(0, "\0")

In [9]:
print (''.join(chars_))


 
 !"#$%&'()*+,-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~����������������������������������������������������������������������


In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars_))
indices_char = dict((i, c) for i, c in enumerate(chars_))

tilde_idx = chars_.index('~')+1
chars = chars_[:tilde_idx]
print (''.join(chars))

vocab_size = len(chars)
print('total chars:', vocab_size)

import pickle
pickle.dump(char_indices, open("data/trump_char_indices.pkl", "w"))
pickle.dump(indices_char, open("data/trump_indices_char.pkl", "w"))
pickle.dump(chars, open("data/trump_chars.pkl", "w"))


 
 !"#$%&'()*+,-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~
total chars: 95


In [10]:
idx = [char_indices[c] for c in text]
print ("before clipping: vocab size is ", max(idx))
idx = np.array(idx).clip(0, tilde_idx-1)
print ("after clipping: vocab size is ", max(idx))


before clipping: vocab size is  164
after clipping: vocab size is  94


In [11]:
print (len(idx))
idx[:10]
idx = idx[:1000000]


3306584


In [12]:
''.join(indices_char[i] for i in idx[:70])

"Jackie Evancho's album sales have skyrocketed after announcing her Ina"

## Preprocess and create model

In [13]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 999961


In [14]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [15]:
sentences.shape, next_chars.shape

((999959, 40), (999959, 40))

In [16]:
n_fac = 24

In [17]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/topology.py:368: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


In [18]:
model.load_weights("data/trump_epoch0_0.000.h5")

## Train

In [19]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(2000):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    return seed_string[40:]

In [20]:
tweets = list()
for lr in [1e-4, 1e-5]:
    for epoch in [0, 1]:
        model.optimizer.lr=lr
        model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)
        model.save_weights('data/trump_epoch%d_%3.6f.h5' % (epoch, lr))
    tweets.append (print_example())

Epoch 1/1
999959/999959 [==============================] - 1858s - loss: 1.7404  
Epoch 1/1
999959/999959 [==============================] - 1857s - loss: 1.5587  
Epoch 1/1
999959/999959 [==============================] - 1857s - loss: 1.4956  
Epoch 1/1
999959/999959 [==============================] - 1857s - loss: 1.4566  


In [21]:
tweets = print_example()

In [22]:
print (tweets)

 at never mention DonaldTrump has asking worse, great speech and vets at oor with Like Aberica great again." She can't be am including Nov over the people of Nc government."
~~~@realDonaldTrump Rink who is about my great?
Via @bredlerding Olama was going down and special interests made voters past. Good luck we got the reduce w/ OK what you're making my support Donald Trump
Thank you for you! #MakeAmericaGreatAgain"
"@GarkeOSson: @realDonaldTrump I think low lost!
Thank you @tedcruz can't win in fistried to give $2$. Clinton. What is in order to say that I don't survived reason doing in Iowa. When they would love @marcorubio. Watching it!
"@AjboryGen: @megynkelly
Hillary looks like social interests to the Saturday at 11:30. A win- he went his ording!
I will be in Carallove, who are they never say Wondly and the borders are speaking a true mention that they would be forgotten, now days last night - 10%!
Thank you New Hampshire! #trump2016 #MAGA https://t.co/3NfOAqLtSj
Iowa made sure a s

In [23]:
model.save_weights("data/trump_final_data.h5")

j = model.to_json()
f = open("data/trump_model.json", "w")
f.write(j)
f.close()

# Now, go look at trump-generate.ipynb